In [2]:
from bs4 import BeautifulSoup
import pandas as pd
from splinter import Browser
from pprint import pprint
from datetime import datetime
import requests
import time

# MARS NEWS

In [3]:
# url
url = 'https://mars.nasa.gov/news/'


In [4]:
# Retrieve page with the requests module
response = requests.get(url)


In [5]:
# Create BeautifulSoup object; parse with 'lxml'
soup = BeautifulSoup(response.text, 'lxml')

In [6]:
# Executable Path/Initialize Browser
executable_path = {"executable_path": "/usr/local/bin/chromedriver"}
browser = Browser("chrome", **executable_path, headless=False)


In [7]:
# Scrape the Latest News Title
news_title = soup.find("div", class_="content_title").get_text()
print(news_title)

# Scrape the Latest Paragraph Text
news_p = soup.find("div", class_="rollover_description_inner").get_text()
print(news_p)



NASA's Perseverance Mars Rover Gets Its Wheels and Air Brakes



After the rover was shipped from JPL to Kennedy Space Center, the team is getting closer to finalizing the spacecraft for launch later this summer.



# MARS IMAGES

In [8]:
# JPL Mars Space Images - Featured Image

executable_path = {"executable_path": "/usr/local/bin/chromedriver"}
browser = Browser("chrome", **executable_path, headless=False)
url = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
browser.visit(url)

In [9]:
## Use splinter to navigate the site and find the image url for the current Featured Mars Image and assign the url string to a variable called `featured_image_url`.

full_image_button = browser.find_by_id("full_image")
full_image_button.click()

In [10]:
# Find More Info button to click
browser.is_element_present_by_text("more info", wait_time=1)
more_info = browser.find_link_by_partial_text("more info")
more_info.click()

/Users/georgialeigh/opt/anaconda3/lib/python3.7/site-packages/splinter/driver/webdriver/__init__.py:528: FutureWarning: browser.find_link_by_partial_text is deprecated. Use browser.links.find_by_partial_text instead.
  FutureWarning,


In [11]:
# Parse Results HTML with BeautifulSoup
html = browser.html
image_soup = BeautifulSoup(html, "html.parser")

In [12]:
# Image URL
img_url = image_soup.select_one("figure.lede a img").get("src")
featured_image_url=(url + img_url)
# print(featured_image_url)


# MARS WEATHER

In [13]:
## 

html = requests.get("https://twitter.com/marswxreport?lang=en").text
weather_soup = BeautifulSoup(html, "html.parser")

try:
    tweet = weather_soup.find_all('div', class_="js-tweet-text-container")
    i = 0
    for tweets in tweet:
        if "InSight" in tweet[i].text:
            print("mars_weather = " + tweet[i].text.split("pic")[0])
            mars_weather="mars_weather = " + tweet[i].text.split("pic")[0]
            break
        i += 1
except:
    print("tweet not found")
    mars_weather=""
    


mars_weather = 
InSight sol 484 (2020-04-06) low -94.3ºC (-137.8ºF) high -7.0ºC (19.3ºF)
winds from the SW at 5.4 m/s (12.1 mph) gusting to 17.3 m/s (38.7 mph)
pressure at 6.50 hPa


# MARS FACTS

In [14]:
# Visit the Mars Facts webpage [here](https://space-facts.com/mars/) and use Pandas to scrape the table containing facts about the planet including Diameter, Mass, etc.
mars_df = pd.read_html("https://space-facts.com/mars/")[0]
# print(mars_df)
# Use Pandas to convert the data to a HTML table string.
mars_df.columns=["Mars Facts", "Data"]
mars_facts=mars_df.to_html()

# MARS HEMISPHERES

In [15]:
# Visit the USGS Astrogeology site [here](https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars) to obtain high resolution images for each of Mar's hemispheres.

executable_path = {"executable_path": "/usr/local/bin/chromedriver"}
browser = Browser("chrome", **executable_path, headless=False)
url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
browser.visit(url)

In [16]:
hemisphere_image_urls = []

# Get a List of All the Hemispheres
links = browser.find_by_css("a.product-item h3")
for item in range(len(links)):
    hemisphere = {}
    
    browser.find_by_css("a.product-item h3")[item].click()
    sample_element = browser.find_by_text("Sample").first
    hemisphere["img_url"] = sample_element["href"]
    hemisphere["title"] = browser.find_by_css("h2.title").text
    
    # Add Hemisphere URL to List
    hemisphere_image_urls.append(hemisphere)
    
    # Go Back
    browser.back()
    
hemisphere_image_urls

[{'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg',
  'title': 'Cerberus Hemisphere Enhanced'},
 {'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg',
  'title': 'Schiaparelli Hemisphere Enhanced'},
 {'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg',
  'title': 'Syrtis Major Hemisphere Enhanced'},
 {'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg',
  'title': 'Valles Marineris Hemisphere Enhanced'}]

In [17]:
results = dict({'hemisphere_image_urls': hemisphere_image_urls,
               'mars_facts': mars_facts,
               'mars_weather': mars_weather,
               'feature_image_url': featured_image_url,
               'new_title': news_title,
               'news_p': news_p})
results

{'hemisphere_image_urls': [{'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg',
   'title': 'Cerberus Hemisphere Enhanced'},
  {'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg',
   'title': 'Schiaparelli Hemisphere Enhanced'},
  {'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg',
   'title': 'Syrtis Major Hemisphere Enhanced'},
  {'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg',
   'title': 'Valles Marineris Hemisphere Enhanced'}],
 'mars_facts': '<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Mars Facts</th>\n      <th>Data</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>0</th>\n      <td>Equatorial Diameter:</td>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <th>1</t